# MSDS 692 Practicum 1 
## Mary Hollon 2-24-2025

### Notebook Objective:

 - Created tables in my database censusdb
 - Load the data
 - Make data checks using MySQL queries.

In [ ]:
#!pip install tabulate
# This makes the table output more readable

In [46]:
import mysql.connector
import csv
import pandas as pd



In [2]:
from getpass import getpass
from mysql.connector import connect, Error
from datetime import datetime

### Create a database connection object 

In [3]:
try:
  mydb = connect(
    user='root',    # could be root, or a user you created
    passwd=getpass('Enter password:'),  # the password for that you used
    database='censusdb',   # the database to connect to
    host='127.0.0.1',   # localhost
    # database='boating', # (optional) database to connect to
    allow_local_infile=True  # needed so can load local files
  )
except Error as e:
  print(e)

Enter password: ········


## Check if connection is established¶


In [4]:
print(mydb.is_connected())


True


## Create  Database Cursor to Execute DB Commands

cursor = mydb.cursor()

cursor.execute ("show databases")`

for x in cursor:
    print(x) 

In [5]:
cursor = mydb.cursor()

In [6]:
# We need to reset the variable that allows loading of local files 
cursor.execute('set global local_infile = 1')

## Let's look at all available databases in MySQL:

In [7]:
cursor.execute ("show databases")  
for x in cursor:
	print(x[0]) 

censusdb
companydata
fitclubdb
information_schema
mysql
performance_schema
sailor_db_week3
sakila
sys
test
week2
world


### Specify the database to use:

In [8]:
cursor.execute('use censusdb')

In [9]:
# Create Tables for Census database censusdb

# Create Demographics Table

cursor.execute("""CREATE TABLE IF NOT EXISTS Demographics (
    state VARCHAR(50),
    year INT,
    total_population INT,
    median_age FLOAT,
    male_population INT,
    female_population INT,
    white_population INT,
    black_population INT,
    asian_population INT,
    hispanic_population INT,
    veteran_status INT,
    foreign_born_population INT,
    non_english_speakers INT,
    PRIMARY KEY (state, year)
)""");



In [10]:
# print out which tables are in censusdb
cursor.execute ("show tables") 
for x in cursor:
	print(x) 

('demographics',)
('education',)
('health_insurance',)
('housing',)
('income',)
('transportation',)


In [11]:
from tabulate import tabulate

cursor.execute("DESCRIBE demographics;")
columns = cursor.fetchall()

# Convert the list of tuples into a list of lists for tabulate
table = [list(col) for col in columns]

# Define column headers
headers = ["Field", "Type", "Null", "Key", "Default", "Extra"]

# Print as a formatted table
print(tabulate(table, headers=headers, tablefmt="grid"))


+-------------------------+-------------+--------+-------+-----------+---------+
| Field                   | Type        | Null   | Key   | Default   | Extra   |
+=========================+=============+========+=======+===========+=========+
| state                   | varchar(50) | NO     | PRI   |           |         |
+-------------------------+-------------+--------+-------+-----------+---------+
| year                    | int         | NO     | PRI   |           |         |
+-------------------------+-------------+--------+-------+-----------+---------+
| total_population        | int         | YES    |       |           |         |
+-------------------------+-------------+--------+-------+-----------+---------+
| median_age              | float       | YES    |       |           |         |
+-------------------------+-------------+--------+-------+-----------+---------+
| male_population         | int         | YES    |       |           |         |
+-------------------------+-

### Let's Create the Remaining Tables: 

In [12]:
# Create Income Table

cursor.execute("""CREATE TABLE IF NOT EXISTS Income (
    state VARCHAR(50),
    year INT,
    median_household_income INT,
    per_capita_income INT,
    population_in_poverty INT,
    total_population_for_poverty INT,
    unemployed_population INT,
    labor_force_population INT,
    PRIMARY KEY (state, year),
    FOREIGN KEY (state, year) REFERENCES Demographics(state, year)
)""");

# Create Housing Table
cursor.execute("""CREATE TABLE IF NOT EXISTS Housing (
    state VARCHAR(50),
    year INT,
    total_households INT,
    owner_occupied_homes INT,
    renter_occupied_homes INT,
    median_home_value INT,
    median_rent INT,
    cost_burdened_households INT,
    vacancy_rate FLOAT,
    average_household_size FLOAT,
    PRIMARY KEY (state, year),
    FOREIGN KEY (state, year) REFERENCES Demographics(state, year)
)""");

# Create Transportation Table
cursor.execute("""CREATE TABLE IF NOT EXISTS Transportation (
    state VARCHAR(50),
    year INT,
    mean_travel_time FLOAT,
    car_commute INT,
    public_transit_commute INT,
    walk_commute INT,
    work_from_home INT,
    PRIMARY KEY (state, year),
    FOREIGN KEY (state, year) REFERENCES Demographics(state, year)
)""");

# Create Health Insurance Table
cursor.execute("""CREATE TABLE IF NOT EXISTS Health_Insurance (
    state VARCHAR(50),
    year INT,
    total_insured_population INT,
    total_uninsured_population INT,
    private_insurance INT,
    public_insurance INT,
    no_health_insurance INT,
    PRIMARY KEY (state, year),
    FOREIGN KEY (state, year) REFERENCES Demographics(state, year)
)""");

# Create Education Table
cursor.execute("""CREATE TABLE IF NOT EXISTS Education (
    state VARCHAR(50),
    year INT,
    total_population_25_over INT,
    less_than_hs INT,
    high_school_graduate INT,
    some_college INT,
    associates_degree INT,
    bachelors_degree INT,
    graduate_degree INT,
    PRIMARY KEY (state, year),
    FOREIGN KEY (state, year) REFERENCES Demographics(state, year)
)""");


##  Verify Table Creation

In [13]:
#Let's look at the tables to make sure they exist in the database

# print out which tables are in censusdb

cursor.execute ("show tables") 
for x in cursor:
	print(x) 

('demographics',)
('education',)
('health_insurance',)
('housing',)
('income',)
('transportation',)


#### The desired tables are all here. Next, let's verify their structures 

In [14]:
# List of table names
tables = ["education", "health_insurance", "housing", "income", "transportation"]

for table_name in tables:
    cursor.execute(f"DESCRIBE {table_name};")
    columns = cursor.fetchall()

    # Convert the list of tuples into a list of lists for tabulate
    table = [list(col) for col in columns]

    # Define column headers
    headers = ["Field", "Type", "Null", "Key", "Default", "Extra"]

    # Print table name as a header
    print(f"\nSchema for table: {table_name}")
    print(tabulate(table, headers=headers, tablefmt="grid"))



Schema for table: education
+--------------------------+-------------+--------+-------+-----------+---------+
| Field                    | Type        | Null   | Key   | Default   | Extra   |
+==========================+=============+========+=======+===========+=========+
| state                    | varchar(50) | NO     | PRI   |           |         |
+--------------------------+-------------+--------+-------+-----------+---------+
| year                     | int         | NO     | PRI   |           |         |
+--------------------------+-------------+--------+-------+-----------+---------+
| total_population_25_over | int         | YES    |       |           |         |
+--------------------------+-------------+--------+-------+-----------+---------+
| less_than_hs             | int         | YES    |       |           |         |
+--------------------------+-------------+--------+-------+-----------+---------+
| high_school_graduate     | int         | YES    |       |          

### The table structures are verified. Next Load the data.

In [16]:
import os
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\mjhol\OneDrive\MSDE 631 SQL_NoSQL\Week4_MySQL_Python_Connector


In [17]:
#"C:\Users\mjhol\OneDrive\MSDS 692 Practicum 1\acs_data_2013.csv"

os.chdir("C:/Users/mjhol/OneDrive/MSDS 692 Practicum 1") 

Because All of the Table Data was pulled as one big file, I have to break up the data according to each table's parameters. The years and The table Columns are defined before loading the data.

In [18]:

import pandas as pd

# Database Connection
cursor = mydb.cursor()

# List of years
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

# Define table column mappings
table_columns = {
    "Demographics": ["State", "Year", "Total_Population", "Median_Age", "Male_Population", 
                     "Female_Population", "White_Population", "Black_Population", 
                     "Asian_Population", "Hispanic_Population", "Veteran_Status", 
                     "Foreign_Born_Population", "Non_English_Speakers"],

    "Income": ["State", "Year", "Median_Household_Income", "Per_Capita_Income", 
               "Population_in_Poverty", "Total_Population_for_Poverty", 
               "Unemployed_Population", "Labor_Force_Population"],

    "Housing": ["State", "Year", "Total_Households", "Owner_Occupied_Homes", 
                "Renter_Occupied_Homes", "Median_Home_Value", "Median_Rent", 
                "Cost_Burdened_Households", "Vacancy_Rate", "Average_Household_Size"],

    "Transportation": ["State", "Year", "Mean_Travel_Time", "Car_Commute", 
                       "Public_Transit_Commute", "Walk_Commute", "Work_From_Home"],

    "Health_Insurance": ["State", "Year", "Total_Insured_Population", 
                         "Total_Uninsured_Population", "Private_Insurance", 
                         "Public_Insurance", "No_Health_Insurance"],

    "Education": ["State", "Year", "Total_Population_25_Over", "Less_Than_HS", 
                  "High_School_Graduate", "Some_College", "Associates_Degree", 
                  "Bachelors_Degree", "Graduate_Degree"]
}

# Function to insert data
def insert_data(table_name, dataframe):
    placeholders = ", ".join(["%s"] * len(dataframe.columns))
    columns = ", ".join(dataframe.columns)
    query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    for row in dataframe.itertuples(index=False, name=None):
        cursor.execute(query, row)

    mydb.commit()
    print(f"✅ Data inserted into {table_name} for {dataframe['Year'].iloc[0]}")

# Loop through each year and process data
for year in years:
    file_path = f"acs_data_{year}.csv"

    # Check if file exists
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        continue
    
    print(f"📂 Processing file: {file_path}")

    # Load CSV
    df = pd.read_csv(file_path)

    # Ensure proper formatting
    df["State"] = df["State"].str.strip()
    df["Year"] = df["Year"].astype(int)

    # Insert data into each table
    for table, cols in table_columns.items():
        table_df = df[cols]
        insert_data(table, table_df)

# Close the connection
cursor.close()
mydb.close()

print("🎉 All data successfully loaded into MySQL!")


📂 Processing file: acs_data_2013.csv
✅ Data inserted into Demographics for 2013
✅ Data inserted into Income for 2013
✅ Data inserted into Housing for 2013
✅ Data inserted into Transportation for 2013
✅ Data inserted into Health_Insurance for 2013
✅ Data inserted into Education for 2013
📂 Processing file: acs_data_2014.csv
✅ Data inserted into Demographics for 2014
✅ Data inserted into Income for 2014
✅ Data inserted into Housing for 2014
✅ Data inserted into Transportation for 2014
✅ Data inserted into Health_Insurance for 2014
✅ Data inserted into Education for 2014
📂 Processing file: acs_data_2015.csv
✅ Data inserted into Demographics for 2015
✅ Data inserted into Income for 2015
✅ Data inserted into Housing for 2015
✅ Data inserted into Transportation for 2015
✅ Data inserted into Health_Insurance for 2015
✅ Data inserted into Education for 2015
📂 Processing file: acs_data_2016.csv
✅ Data inserted into Demographics for 2016
✅ Data inserted into Income for 2016
✅ Data inserted into H

In [ ]:
#!pip install --upgrade numpy pandas


In [32]:
# Try to connect to the database properly
try:
    mydb = mysql.connector.connect(
        user='root',    
        passwd=getpass('Enter password:'),  
        database='censusdb',   
        host='127.0.0.1',   
        allow_local_infile=True  
    )
    print("✅ Connected to MySQL database")

except Error as e:
    print(f"❌ Error connecting to database: {e}")
    exit()  # Stop execution if connection fails




Enter password: ········


✅ Connected to MySQL database


## Explore The Data To Ensure it Has Loaded Properly

In [33]:
# Create a new cursor
cursor = mydb.cursor()

# Ensure any unread results are handled before running a new query
try:
    while cursor.nextset():
        cursor.fetchall()  # Clear any unread results
except:
    pass  # Ignore errors if no unread results exist

# Execute the query safely
try:
    query = """
        SELECT 'Demographics' AS TableName, COUNT(*) AS RowCount FROM Demographics
        UNION ALL
        SELECT 'Income', COUNT(*) FROM Income
        UNION ALL
        SELECT 'Housing', COUNT(*) FROM Housing
        UNION ALL
        SELECT 'Transportation', COUNT(*) FROM Transportation
        UNION ALL
        SELECT 'Health_Insurance', COUNT(*) FROM Health_Insurance
        UNION ALL
        SELECT 'Education', COUNT(*) FROM Education
    """
    
    cursor.execute(query)
    results = cursor.fetchall()

    # Display the results
    print("\n📊 Table Row Counts:")
    for row in results:
        print(row)

except Error as e:
    print(f"❌ Error executing query: {e}")



📊 Table Row Counts:
('Demographics', 520)
('Income', 520)
('Housing', 520)
('Transportation', 520)
('Health_Insurance', 520)
('Education', 520)


### The Table Row Counts are as Expected.
Next, I will explore the tables to ensure that the data is loaded correctly and as expected.
To Make the output easier to read, I will display the query results in a Pandas Dataframe

In [47]:
query = """SELECT * FROM Demographics LIMIT 20"""
cursor.execute(query)

# Fetch column names
column_names = [desc[0] for desc in cursor.description]

results = cursor.fetchall()

# Convert results into a Pandas DataFrame
df = pd.DataFrame(results, columns=column_names)

# Display the results
df.head(20)


,state,year,total_population,median_age,male_population,female_population,white_population,black_population,asian_population,hispanic_population,veteran_status,foreign_born_population,non_english_speakers
0,Alabama,2013,4799277,38.1,2328592,2470685,3326188,1262152,56831,188294,3660703,168232,4497352
1,Alabama,2014,4817678,38.2,2336020,2481658,3327891,1269808,58322,191838,3685939,166477,4518107
2,Alabama,2015,4830620,38.4,2341093,2489527,3325464,1276544,59599,193492,3705690,167224,4535566
3,Alabama,2016,4841164,38.6,2346193,2494971,3325037,1282053,60744,193503,3723825,165504,4548393
4,Alabama,2017,4850771,38.7,2350806,2499965,3317702,1287167,62815,198375,3735780,167926,4558608
5,Alabama,2018,4864680,38.9,2355799,2508881,3317453,1293186,64609,203146,3753332,168111,4572555
6,Alabama,2019,4876250,39.0,2359355,2516895,3320247,1299048,66270,208626,3766571,172947,4583063
7,Alabama,2020,4893186,39.2,2365734,2527452,3302834,1301319,67909,212951,3785626,168053,4599254
8,Alabama,2021,4997675,39.3,2429703,2567972,3338590,1324233,70161,224659,3859695,173429,4702043
9,Alabama,2022,5028092,39.3,2445995,2582097,3329012,1326341,69808,232407,3902062,177207,4736236


In [48]:
query = """SELECT * FROM Education LIMIT 20"""
cursor.execute(query)

# Fetch column names
column_names = [desc[0] for desc in cursor.description]

results = cursor.fetchall()

# Convert results into a Pandas DataFrame
df = pd.DataFrame(results, columns=column_names)

# Display the results
df.head(20)


,state,year,total_population_25_over,less_than_hs,high_school_graduate,some_college,associates_degree,bachelors_degree,graduate_degree
0,Alabama,2013,3193338,40711,818315,511830,458393,187852,30801
1,Alabama,2014,3217902,41671,824979,514936,465268,196935,31815
2,Alabama,2015,3239351,41876,826272,516724,478812,201997,32332
3,Alabama,2016,3261408,42577,828205,517123,492382,208839,32572
4,Alabama,2017,3276637,41118,831994,513751,503930,216001,34195
5,Alabama,2018,3299958,41240,838776,514121,515443,221842,34881
6,Alabama,2019,3320877,40837,841716,508105,529178,230528,35796
7,Alabama,2020,3344006,41430,839252,512048,546674,239746,38455
8,Alabama,2021,3413803,43813,864345,514689,563628,251040,41681
9,Alabama,2022,3428520,45445,864977,507480,572252,261039,42980


In [53]:
query = """SELECT * FROM Income LIMIT 20"""
cursor.execute(query)

# Fetch column names
column_names = [desc[0] for desc in cursor.description]

results = cursor.fetchall()

# Convert results into a Pandas DataFrame
df1 = pd.DataFrame(results, columns=column_names)

# Display the results
df1.tail(10)

,state,year,median_household_income,per_capita_income,population_in_poverty,total_population_for_poverty,unemployed_population,labor_force_population
10,Alaska,2013,70760,32651,69514,704041,32939,393037
11,Alaska,2014,71829,33129,71866,711235,32097,396856
12,Alaska,2015,72515,33413,72957,716218,31285,399077
13,Alaska,2016,74444,34191,72826,719064,30139,400475
14,Alaska,2017,76114,35065,73380,719983,29548,400622
15,Alaska,2018,76715,35874,77865,720674,28067,396353
16,Alaska,2019,77640,36787,76933,719376,26808,392338
17,Alaska,2020,77790,37094,74369,719445,26429,386787
18,Alaska,2021,80287,39236,75016,719086,25054,383795
19,Alaska,2022,86370,42828,75227,717293,23035,383078


In [45]:
import pandas as pd

query = "SELECT * FROM Transportation LIMIT 20"
cursor.execute(query)

# Fetch column names
column_names = [desc[0] for desc in cursor.description]

# Fetch results
results = cursor.fetchall()

# Convert results into a Pandas DataFrame
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
df.head(20)

,state,year,mean_travel_time,car_commute,public_transit_commute,walk_commute,work_from_home
0,Alabama,2013,46428500.0,1869731,8935,23233,51785
1,Alabama,2014,46805400.0,1877915,8943,22697,53344
2,Alabama,2015,47251900.0,1888257,8452,22288,55668
3,Alabama,2016,47968200.0,1903947,7841,23010,58991
4,Alabama,2017,48314300.0,1916975,7433,22659,62502
5,Alabama,2018,49084900.0,1934423,6959,22977,66881
6,Alabama,2019,49913600.0,1953288,6927,23376,69165
7,Alabama,2020,50359400.0,1953038,6960,23265,92836
8,Alabama,2021,51105500.0,1975704,7097,23088,120723
9,Alabama,2022,51658900.0,1985736,6982,24058,145406


In [54]:
query = """SELECT * FROM Housing LIMIT 20"""
cursor.execute(query)

# Fetch column names
column_names = [desc[0] for desc in cursor.description]

results = cursor.fetchall()

# Convert results into a Pandas DataFrame
df = pd.DataFrame(results, columns=column_names)

# Display the results
df.head(10)

,state,year,total_households,owner_occupied_homes,renter_occupied_homes,median_home_value,median_rent,cost_burdened_households,vacancy_rate,average_household_size
0,Alabama,2013,2178116,1281604,557079,122500,705,43229,339433.0,2.55
1,Alabama,2014,2190638,1274196,567978,123800,715,43125,348464.0,2.55
2,Alabama,2015,2199329,1269145,579180,125500,717,44060,351004.0,2.55
3,Alabama,2016,2209335,1267824,583237,128500,728,43058,358274.0,2.55
4,Alabama,2017,2231126,1273217,583478,132100,747,43678,374431.0,2.55
5,Alabama,2018,2244462,1275223,585046,137200,772,43244,384193.0,2.55
6,Alabama,2019,2255026,1284748,583145,142700,792,42905,387133.0,2.55
7,Alabama,2020,2270398,1306505,581999,149600,811,42640,381894.0,2.53
8,Alabama,2021,2278526,1320571,582412,157100,852,44097,375543.0,2.57
9,Alabama,2022,2296920,1347792,585358,179400,925,45062,363770.0,2.53


In [55]:
query = """SELECT * FROM Health_Insurance LIMIT 20"""
cursor.execute(query)

# Fetch column names
column_names = [desc[0] for desc in cursor.description]

results = cursor.fetchall()

# Convert results into a Pandas DataFrame
df = pd.DataFrame(results, columns=column_names)

# Display the results
df.head(10)

,state,year,total_insured_population,total_uninsured_population,private_insurance,public_insurance,no_health_insurance
0,Alabama,2013,4716915,57347,1121332,294979,300398
1,Alabama,2014,4735953,51606,1115913,286339,299742
2,Alabama,2015,4749786,45500,1110011,265669,287141
3,Alabama,2016,4761291,39111,1103328,245685,264957
4,Alabama,2017,4770692,41459,1170330,221678,245506
5,Alabama,2018,4785040,38671,1165772,204561,231101
6,Alabama,2019,4797215,36972,1162995,193275,224687
7,Alabama,2020,4813429,36714,1160135,190423,224992
8,Alabama,2021,4920010,40628,1190423,193735,239179
9,Alabama,2022,4944981,39485,1184371,193271,235651


## Next, Check For Duplicate Rows by State and Year

In [57]:
query = """SELECT State, Year, COUNT(*) AS RowCount
FROM Demographics
GROUP BY State, Year
HAVING COUNT(*) > 1"""

cursor.execute(query)
results = cursor.fetchall()

# Print results
print("\n Duplicate Records in Demographics:")
for row in results:
    print(row)



 Duplicate Records in Demographics:


In [58]:
query = """SELECT State, Year, COUNT(*) AS RowCount
FROM Income
GROUP BY State, Year
HAVING COUNT(*) > 1"""

cursor.execute(query)
results = cursor.fetchall()

# Print results
print("\n Duplicate Records in Income:")
for row in results:
    print(row)


 Duplicate Records in Income:


In [59]:
query = """SELECT State, Year, COUNT(*) AS RowCount
FROM Housing
GROUP BY State, Year
HAVING COUNT(*) > 1"""

cursor.execute(query)
results = cursor.fetchall()

# Print results
print("\n Duplicate Records in Housing:")
for row in results:
    print(row)


 Duplicate Records in Housing:


In [60]:
query = """SELECT State, Year, COUNT(*) AS RowCount
FROM Transportation
GROUP BY State, Year
HAVING COUNT(*) > 1"""

cursor.execute(query)
results = cursor.fetchall()

# Print results
print("\n Duplicate Records in Transportation:")
for row in results:
    print(row)


 Duplicate Records in Transportation:


In [61]:
query = """SELECT State, Year, COUNT(*) AS RowCount
FROM Health_Insurance
GROUP BY State, Year
HAVING COUNT(*) > 1"""

cursor.execute(query)
results = cursor.fetchall()

# Print results
print("\n Duplicate Records in Health_Insurance:")
for row in results:
    print(row)


 Duplicate Records in Health_Insurance:


In [62]:
query = """SELECT State, Year, COUNT(*) AS RowCount
FROM Education
GROUP BY State, Year
HAVING COUNT(*) > 1"""

cursor.execute(query)
results = cursor.fetchall()

# Print results
print("\n Duplicate Records in Education:")
for row in results:
    print(row)


 Duplicate Records in Education:


### None of the tables contain duplicate rows

### Spot Check For NULL Values in the Data

In [74]:
query = """
    SELECT State, Year, Total_Population, Median_Age 
    FROM Demographics 
    WHERE Total_Population IS NULL OR Median_Age IS NULL 
    ORDER BY Year, State
    LIMIT 10
"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

In [75]:
query = """
    SELECT State, Year, Median_Household_Income, Population_in_Poverty
    FROM Income
    WHERE Median_Household_Income IS NULL OR Population_in_Poverty IS NULL 
    ORDER BY Year, State
    LIMIT 10
"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

In [76]:
query = """
    SELECT State, Year, Total_Households, Median_Home_Value
    FROM Housing
    WHERE Total_Households IS NULL OR  Median_Home_Value IS NULL 
    ORDER BY Year, State
    LIMIT 10
"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

In [77]:
query = """
    SELECT State, Year, Mean_Travel_Time, Car_Commute
    FROM Transportation
    WHERE Mean_Travel_Time IS NULL OR  Car_Commute IS NULL 
    ORDER BY Year, State
    LIMIT 10
"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

In [78]:
query = """
    SELECT State, Year, Total_Insured_Population, No_Health_Insurance
    FROM Health_Insurance
    WHERE Total_Insured_Population IS NULL OR  No_Health_Insurance IS NULL 
    ORDER BY Year, State
    LIMIT 10
"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

In [79]:
query = """
    SELECT State, Year, Total_Population_25_Over , Bachelors_Degree
    FROM Education
    WHERE Total_Population_25_Over  IS NULL OR  Bachelors_Degree IS NULL 
    ORDER BY Year, State
    LIMIT 10
"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

### The Spot Checks Look Good

### Perform One More Check For Distinct States for 2 of the tables


In [73]:
query = """
SELECT DISTINCT State FROM Demographics
ORDER BY State
"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)


('Alabama',)
('Alaska',)
('Arizona',)
('Arkansas',)
('California',)
('Colorado',)
('Connecticut',)
('Delaware',)
('District of Columbia',)
('Florida',)
('Georgia',)
('Hawaii',)
('Idaho',)
('Illinois',)
('Indiana',)
('Iowa',)
('Kansas',)
('Kentucky',)
('Louisiana',)
('Maine',)
('Maryland',)
('Massachusetts',)
('Michigan',)
('Minnesota',)
('Mississippi',)
('Missouri',)
('Montana',)
('Nebraska',)
('Nevada',)
('New Hampshire',)
('New Jersey',)
('New Mexico',)
('New York',)
('North Carolina',)
('North Dakota',)
('Ohio',)
('Oklahoma',)
('Oregon',)
('Pennsylvania',)
('Puerto Rico',)
('Rhode Island',)
('South Carolina',)
('South Dakota',)
('Tennessee',)
('Texas',)
('Utah',)
('Vermont',)
('Virginia',)
('Washington',)
('West Virginia',)
('Wisconsin',)
('Wyoming',)


In [80]:
query = """
SELECT DISTINCT State FROM Education
ORDER BY State
"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(row)

('Alabama',)
('Alaska',)
('Arizona',)
('Arkansas',)
('California',)
('Colorado',)
('Connecticut',)
('Delaware',)
('District of Columbia',)
('Florida',)
('Georgia',)
('Hawaii',)
('Idaho',)
('Illinois',)
('Indiana',)
('Iowa',)
('Kansas',)
('Kentucky',)
('Louisiana',)
('Maine',)
('Maryland',)
('Massachusetts',)
('Michigan',)
('Minnesota',)
('Mississippi',)
('Missouri',)
('Montana',)
('Nebraska',)
('Nevada',)
('New Hampshire',)
('New Jersey',)
('New Mexico',)
('New York',)
('North Carolina',)
('North Dakota',)
('Ohio',)
('Oklahoma',)
('Oregon',)
('Pennsylvania',)
('Puerto Rico',)
('Rhode Island',)
('South Carolina',)
('South Dakota',)
('Tennessee',)
('Texas',)
('Utah',)
('Vermont',)
('Virginia',)
('Washington',)
('West Virginia',)
('Wisconsin',)
('Wyoming',)


### The Output is as Expected. These queries returned a list of distinct state names 

### Close the database 

In [83]:
# Close the cursor and database connection safely
# Close the connection
cursor.close()
mydb.close()
print("✅ Database connection closed.")


✅ Database connection closed.


## END OF NOTEBOOK

This Notebook created tables in my database censusdb, loaded the data, and made data checks using MySQL queries as stated in the notebook objective.